In [2]:
#This class defines operators
class Operator:
    action = -1
    
    def __init__(self, act):
        self.action = act

#This class defines state 
class State:
    # This variable stores the tile configuration
    tiledArr = numpy.empty((3,3),dtype='int')
    
    # These two variables store the current location of empty tile
    freeTileI = 0
    freeTileJ = 0
    
    # This variable is to retrieve the path after finding a solution
    previous = None
    
    # This stores the cost from start node
    gVal = 0
    
    #Initializing the state with a given a tile configuration (arr)
    def __init__(self, arr):
        self.tiledArr = numpy.empty((3,3),dtype='int')
        for i in range(3):
            for j in range(3):
                self.tiledArr[i][j] = arr[i][j]
                
                #Empty tile is represented as a -1 in the 2D array
                if(self.tiledArr[i][j] == -1):
                    self.freeTileI = i
                    self.freeTileJ = j
    
    #This function returns a 1 if two tile configurations are the same and 0 otherwise
    def isEqual(self, s):
        for i in range(3):
            for j in range(3):
                if(self.tiledArr[i][j] != s.tiledArr[i][j]):
                    return 0
        return 1
    
    #This function applies an operator O on the current state (self) configuration 
    #and returns a new state configuration s1
    def applyOperator(self, o):
        s1 = State(self.tiledArr)
        s1.previous = self
        s1.gVal = self.gVal + 1
        #0 - West, 1 - South, 2 - East and 3 - North'
        if(o.action == 0):
            if(self.freeTileJ - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ-1]
                s1.freeTileJ -= 1
        elif(o.action == 1):
            if(self.freeTileI + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI+1][self.freeTileJ]
                s1.freeTileI += 1
        elif(o.action == 2):
            if(self.freeTileJ + 1 < 3):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI][self.freeTileJ+1]
                s1.freeTileJ += 1
        elif(o.action == 3):
            if(self.freeTileI - 1 >= 0):
                s1.tiledArr[self.freeTileI][self.freeTileJ] = self.tiledArr[self.freeTileI-1][self.freeTileJ]
                s1.freeTileI -= 1
        
        s1.tiledArr[s1.freeTileI][s1.freeTileJ] = -1
                
        return s1
    
    #This function prints the state configuration
    def printState(self):
        for i in range(3):
            for j in range(3):
                print("%d"%self.tiledArr[i][j],end=' ')
            print()
        print("*******")

#  Utility Functions

Next we define some utility functions which can be used by Heuristic Search algorithms. These functions helps in checking whether a state is present in a given list, to retrieve path from a given state.


In [3]:
#Function that indicates whether a state is present in a list
def isPresentStateInList(state, searchList):
    for elem in searchList:
        if(state.isEqual(elem) == 1):
            return 1

    return 0

#Function that indicates whether a state is present in a priority list
def isPresentStateInPriorityList(state, searchList):
    for [elem, dist] in searchList:
        if(state.isEqual(elem) == 1):
            return 1
        
    return 0

def insertStateInPriorityQueue(searchList, state, distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(distanceToGoal < searchList[i][1]):
            index = i
            break
    
    if(len(searchList) == 0 or index == -1):
        searchList.append([state,distanceToGoal])
    else:
        searchList.insert(index, [state,distanceToGoal])

#Reinserts the element in priority queue if the new value is less than the 
#value present in queue.
def checkAndUpdateStateInPriorityQueue(searchList,state,distanceToGoal):
    index = -1
    for i in range(len(searchList)):
        if(state.isEqual(searchList[i][0])):
            if distanceToGoal < searchList[i][1]:
                    index = i
            break
            
    if index!=-1:
        searchList.remove([searchList[index][0],searchList[index][1]])
        insertStateInPriorityQueue(searchList, state, distanceToGoal)
        

        
        
def retrievePathFromState(state):
    visitedstatelist = []
    visitedstatelist.append(state)
    state.printState()
    prev = state.previous
    counter = 0
    while(prev != None):
        visitedstatelist.append(prev)
        #prev.printState()
        prev = prev.previous
        counter +=1
    visitedstatelist.reverse()
    for i in range(len(visitedstatelist)):
        visitedstatelist[i].printState()
    print("Size of path is ",counter,state.gVal)

# Heuristic functions:

In [4]:
# Hamming distance
def calculateHamming(current, goal):
    counter = 0
    for i in range(3):
        for j in range(3):
            if(current.tiledArr[i][j] != goal.tiledArr[i][j]):
                counter+=1

    return counter

# Manhattan distance
def calculateManhattan(current, goal):
    xposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposcurrent = [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    xposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    yposgoal =  [-1,-1,-1,-1,-1,-1,-1,-1,-1]
    for i in range(3):
        for j in range(3):
            if current.tiledArr[i][j]!=-1:
                xposcurrent[current.tiledArr[i][j]] = i
                yposcurrent[current.tiledArr[i][j]] = j
            if goal.tiledArr[i][j]!=-1:
                xposgoal[goal.tiledArr[i][j]] = i
                yposgoal[goal.tiledArr[i][j]] = j
    totdist=0
    for i in range(1,9):
        totdist+= (math.fabs(xposcurrent[i]-xposgoal[i])+math.fabs(yposcurrent[i]-yposgoal[i]))
    
    return totdist